# Inicio
Iremos usar um modelo de predição do Darts para manipular séries temporais.

In [1]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import ExponentialSmoothing
import matplotlib.pyplot as plt
from darts.datasets import ETTh2Dataset
from darts.ad import KMeansScorer
from darts.ad import QuantileDetector
import yfinance as yf
from datetime import datetime

/home/operador/Documentos/Estudos/Projetos/ScienceMarket_Original/.venv/lib/python3.12/site-packages/lightning_fabric/__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)
/home/operador/Documentos/Estudos/Projetos/ScienceMarket_Original/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Importando dados

In [2]:
codigo = 'BRL=X'
inicio = '2000-01-01'
fim = pd.to_datetime(datetime.now()).strftime('%Y-%m-%d')

data = yf.download(codigo, start=inicio, end=fim, multi_level_index=False)

/tmp/ipykernel_160570/117030692.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(codigo, start=inicio, end=fim, multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


## Tratamento

In [3]:
pd.DataFrame(data)[['Close']].reset_index()

,Date,Close
0,2003-12-01,2.9230
1,2003-12-02,2.9310
2,2003-12-03,2.9310
3,2003-12-04,2.9430
4,2003-12-05,2.9340
...,...,...
5230,2025-09-08,5.4122
5231,2025-09-09,5.4203
5232,2025-09-10,5.4336
5233,2025-09-11,5.4020


In [4]:
df = pd.DataFrame({
    'ds': pd.to_datetime(pd.DataFrame(data)[['Close']].reset_index()['Date']),
    'y': pd.DataFrame(data)[['Close']].reset_index()['Close'].astype(float)
})
df = df.dropna()
df = df.sort_values('ds').reset_index(drop=True)

series = TimeSeries.from_dataframe(df, 'ds', 'y', freq='D')

split = int(len(series) * 0.8)
train, val = series[:split], series[split:]
print(f"Dados separados!\nTrain: {len(train)}\nTest: {len(val)}")

Dados separados!
Train: 6365
Test: 1592


In [ ]:
# Os valores estão zerados, tem que retirar os valores nulos.

# Fill NaN values from train using interpolation
train_clean = train.fill_missing_values()

model = ExponentialSmoothing()
model.fit(train_clean)
prediction = model.predict(len(val), num_samples=1000)

AttributeError: 'TimeSeries' object has no attribute 'interpolate'

## Visualização

## Modelo